In [ ]:
import os

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_classic.chains.combine_documents import (
            create_stuff_documents_chain,
        )


In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
url = "https://365datascience.com/courses"
loader = WebBaseLoader(url)
raw_documents = loader.load()


In [ ]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(raw_documents)

In [ ]:
embedings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [ ]:
vectorstore = FAISS.from_documents(documents, embedings)

In [ ]:
history_store = {}

def get_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in history_store:
        history_store[session_id] = InMemoryChatMessageHistory()
    return history_store[session_id]

llm = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0, model="gpt-3.5-turbo", max_tokens=200)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Rewrite the user's question to be standalone based on the chat history."),
        MessagesPlaceholder("chat_history"),
        ("user", "{input}"),
    ]
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the question using the provided context."),
        MessagesPlaceholder("chat_history"),
        ("user", "Context:\n{context}\n\nQuestion: {input}"),
    ]
)

retriever = vectorstore.as_retriever()
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)
qa_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

chat_with_history = RunnableWithMessageHistory(
    rag_chain,
    get_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
query = "What courses are available on this page?"
session_id = "user_1234"
response = chat_with_history.invoke(
    {"input": query},
    config={"configurable": {"session_id": session_id}},
)
print(response["answer"])